<img src="https://cdn.comet.ml/img/notebook_logo.png">

[Comet](https://www.comet.com/site/products/ml-experiment-tracking/) is an MLOps Platform that is designed to help Data Scientists and Teams build better models faster! Comet provides tooling to track, Explain, Manage, and Monitor your models in a single place! It works with Jupyter Notebooks and Scripts and most importantly it's 100% free to get started!

[Composer](https://github.com/mosaicml/composer/tree/dev) is an open-source deep learning training library by [MosaicML](https://www.mosaicml.com/). Built on top of PyTorch, the Composer library makes it easier to implement distributed training workflows on large-scale clusters.

Instrument Composer with Comet to start managing experiments, create dataset versions and track hyperparameters for faster and easier reproducibility and collaboration.

[Find more information about our integration with Composer](https://www.comet.ml/docs/v2/integrations/ml-frameworks/composer/)

Curious about how Comet can help you build better models, faster? Find out more about [Comet](https://www.comet.com/site/products/ml-experiment-tracking/) and our [other integrations](https://www.comet.com/docs/v2/integrations/overview/)

Get a preview for what's to come. Check out a completed experiment created from this notebook [here](TODO).


# Install Dependencies

In [1]:
%pip install "composer>=0.16.1" "comet_ml>=3.33.10"

  Using cached comet_ml-3.43.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached torchmetrics-1.3.2-py3-none-any.whl.metadata (19 kB)
  Using cached torch_optimizer-0.3.0-py3-none-any.whl.metadata (55 kB)
  Using cached torchvision-0.18.1-cp310-cp310-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached coolname-2.2.0-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached importlib_metadata-6.11.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached everett-3.1.0-py2.py3-none-any.whl.metadata (17 kB)
  Using cached python_box-6.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.8 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached semantic_version-2.10.0

# Initialize Comet

In [3]:
import comet_ml

comet_ml.init(project_name="comet-example-mosaicml-getting-started-notebook")

# Import Dependencies

In [10]:
import time

import torch
import torch.utils.data

import composer
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
from composer.loggers import CometMLLogger

torch.manual_seed(42)  # For replicability

# Load Data

In [5]:
data_directory = "./data"

# Normalization constants
mean = (0.507, 0.487, 0.441)
std = (0.267, 0.256, 0.276)

batch_size = 1024

cifar10_transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean, std)]
)

train_dataset = datasets.CIFAR10(
    data_directory, train=True, download=True, transform=cifar10_transforms
)
test_dataset = datasets.CIFAR10(
    data_directory, train=False, download=True, transform=cifar10_transforms
)

# Our train and test dataloaders are PyTorch DataLoader objects!
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 170498071/170498071 [00:06<00:00, 25502895.17it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Define Model

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from composer.models import ComposerClassifier


class Block(nn.Module):
    """A ResNet block."""

    def __init__(self, f_in: int, f_out: int, downsample: bool = False):
        super(Block, self).__init__()

        stride = 2 if downsample else 1
        self.conv1 = nn.Conv2d(
            f_in, f_out, kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(f_out)
        self.conv2 = nn.Conv2d(
            f_out, f_out, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(f_out)
        self.relu = nn.ReLU(inplace=True)

        # No parameters for shortcut connections.
        if downsample or f_in != f_out:
            self.shortcut = nn.Sequential(
                nn.Conv2d(f_in, f_out, kernel_size=1, stride=2, bias=False),
                nn.BatchNorm2d(f_out),
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x: torch.Tensor):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return self.relu(out)


class ResNetCIFAR(nn.Module):
    """A residual neural network as originally designed for CIFAR-10."""

    def __init__(self, outputs: int = 10):
        super(ResNetCIFAR, self).__init__()

        depth = 56
        width = 16
        num_blocks = (depth - 2) // 6

        plan = [(width, num_blocks), (2 * width, num_blocks), (4 * width, num_blocks)]

        self.num_classes = outputs

        # Initial convolution.
        current_filters = plan[0][0]
        self.conv = nn.Conv2d(
            3, current_filters, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn = nn.BatchNorm2d(current_filters)
        self.relu = nn.ReLU(inplace=True)

        # The subsequent blocks of the ResNet.
        blocks = []
        for segment_index, (filters, num_blocks) in enumerate(plan):
            for block_index in range(num_blocks):
                downsample = segment_index > 0 and block_index == 0
                blocks.append(Block(current_filters, filters, downsample))
                current_filters = filters

        self.blocks = nn.Sequential(*blocks)

        # Final fc layer. Size = number of filters in last segment.
        self.fc = nn.Linear(plan[-1][0], outputs)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x: torch.Tensor):
        out = self.relu(self.bn(self.conv(x)))
        out = self.blocks(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


model = ComposerClassifier(module=ResNetCIFAR(), num_classes=10)

# Optimizer and Scheduler

In [7]:
optimizer = composer.optim.DecoupledSGDW(
    model.parameters(),  # Model parameters to update
    lr=0.05,  # Peak learning rate
    momentum=0.9,
    weight_decay=2.0e-3,  # If this looks large, it's because its not scaled by the LR as in non-decoupled weight decay
)

You are using a high value of `weight_decay=0.002` for the `DecoupledSGDW` optimizer. Are you sure you want to do this? Your model's weights will be multiplied by 0.998 on every step!


We'll assume this is being run on Colab, which means training for hundreds of epochs would take a very long time. Instead we'll train our baseline model for three epochs. The first epoch will be linear warmup, followed by two epochs of constant LR. We achieve this by instantiating a `LinearWithWarmupScheduler` class.

**Note**: Composer provides a handful of different [schedulers][schedulers] to help customize your training!

[schedulers]: https://docs.mosaicml.com/projects/composer/en/stable/trainer/schedulers.html

In [8]:
lr_scheduler = composer.optim.LinearWithWarmupScheduler(
    t_warmup="1ep",  # Warm up over 1 epoch
    alpha_i=1.0,  # Flat LR schedule achieved by having alpha_i == alpha_f
    alpha_f=1.0,
)

# Logging to Comet

In [11]:
# "baseline" = no algorithms (which is what we're doing now)
logger_for_baseline = CometMLLogger()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/lothiraldan/comet-example-mosaicml-getting-started/a10fc0bc760a4b41adbaa173fb313763



# Train a Baseline Model

In [12]:
train_epochs = (
    "3ep"  # Train for 3 epochs because we're assuming Colab environment and hardware
)
device = "gpu" if torch.cuda.is_available() else "cpu"  # select the device

trainer = composer.trainer.Trainer(
    model=model,
    train_dataloader=train_dataloader,
    eval_dataloader=test_dataloader,
    max_duration=train_epochs,
    optimizers=optimizer,
    schedulers=lr_scheduler,
    device=device,
    loggers=logger_for_baseline,
)

We train and measure the training time below.

In [13]:
start_time = time.perf_counter()
trainer.fit()  # <-- Your training loop in action!
end_time = time.perf_counter()
print(f"It took {end_time - start_time:0.4f} seconds to train")

******************************
Config:
composer_commit_hash: None
composer_version: 0.23.5
node_name: unknown because NODENAME environment variable not set
num_cpus_per_node: 1
num_nodes: 1
rank_zero_seed: 3481102868

******************************


train          Epoch   0:    0%|| 0/49 [00:00<?, ?ba/s]                                                       …

/home/lothiraldan/.virtualenvs/tempenv-0634243152b7b/lib/python3.10/site-packages/composer/core/data_spec.py:37: UserWarning: Cannot split tensor of length 848 into batches of size 1024. As it is smaller, no splitting will be done. This may happen on the last batch of a dataset if it is a smaller size than the microbatch size.
  warnings.warn(


eval           Epoch   0:    0%|| 0/10 [00:00<?, ?ba/s]                                                       …

/home/lothiraldan/.virtualenvs/tempenv-0634243152b7b/lib/python3.10/site-packages/composer/core/data_spec.py:37: UserWarning: Cannot split tensor of length 784 into batches of size 1024. As it is smaller, no splitting will be done. This may happen on the last batch of a dataset if it is a smaller size than the microbatch size.
  warnings.warn(


train          Epoch   1:    0%|| 0/49 [00:00<?, ?ba/s]                                                       …

eval           Epoch   1:    0%|| 0/10 [00:00<?, ?ba/s]                                                       …

train          Epoch   2:    0%|| 0/49 [00:00<?, ?ba/s]                                                       …

eval           Epoch   2:    0%|| 0/10 [00:00<?, ?ba/s]                                                       …

It took 1405.1789 seconds to train


# Save code to Comet

In [20]:
comet_ml.get_running_experiment().end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : 1720104812-ultraviolet-starling
COMET INFO:     url                   : https://www.comet.com/lothiraldan/comet-example-mosaicml-getting-started/02e438afa6134b8084f704de7aea461c
COMET INFO:   Others:
COMET INFO:     Created from : mosaicml-composer
COMET INFO:     Name         : 1720104812-ultraviolet-starling
COMET INFO:   Parameters:
COMET INFO:     blurpool/num_blurconv_layers : 4
COMET INFO:     blurpool/num_blurpool_layers : 0
COMET INFO:     composer_commit_hash         : None
COMET INFO:     composer_version             : 0.23.5
COMET INFO:     node_name                    : unknown because NODENAME environment variable not set
COMET INFO:    

## Use Algorithms to Speed Up Training
<a id="Use-Algorithms-to-Speed-Up-Training"></a>

One of the things we're most excited about at MosaicML is our arsenal of speed-up [algorithms][algorithms]. We used these algorithms to [speed up training of ResNet-50 on ImageNet by up to 7.6x][explorer]. Let's try applying a few algorithms to make our ResNet-56 more efficient.

Before we jump in, here's a quick primer on Composer speed-up algorithms. Each one is implemented as an `Algorithm` class, which basically just adds some structure that controls what happens when the algorithm is applied and when in the training loop it should be applied. Adding a particular algorithm into the training loop is as simple as creating an instance of it (using args/kwargs to set any hyperparameters) and passing it to the `Trainer` during initialization. We'll see that in action below...

For our first algorithm here, let's start with [Label Smoothing][label_smoothing], which serves as a form of regularization by interpolating between the target distribution and another distribution that usually has higher entropy.

[algorithms]: https://docs.mosaicml.com/projects/composer/en/stable/trainer/algorithms.html
[explorer]: https://app.mosaicml.com/explorer/imagenet
[label_smoothing]: https://docs.mosaicml.com/projects/composer/en/stable/method_cards/label_smoothing.html

In [14]:
label_smoothing = composer.algorithms.LabelSmoothing(
    0.1
)  # We're creating an instance of the LabelSmoothing algorithm class

Let's also use [BlurPool][blurpool], which increases accuracy by applying a spatial low-pass filter before the pool in max pooling and whenever using a strided convolution.

[blurpool]: https://docs.mosaicml.com/projects/composer/en/stable/method_cards/blurpool.html

In [15]:
blurpool = composer.algorithms.BlurPool(
    replace_convs=True,  # Blur before convs
    replace_maxpools=True,  # Blur before max-pools
    blur_first=True,  # Blur before conv/max-pool
)

Our final algorithm in our improved training recipe is [Progressive Image Resizing][progressive_image_resizing]. Progressive Image Resizing initially shrinks the size of training images and slowly scales them back to their full size over the course of training. It increases throughput during the early phase of training, when the network may learn coarse-grained features that do not require the details lost by reducing image resolution.

[progressive_image_resizing]: https://docs.mosaicml.com/projects/composer/en/stable/method_cards/progressive_resizing.html

In [16]:
prog_resize = composer.algorithms.ProgressiveResizing(
    initial_scale=0.6,  # Size of images at the beginning of training = .6 * default image size
    finetune_fraction=0.34,  # Train on default size images for 0.34 of total training time.
)

We'll assemble all our algorithms into a list to pass to our trainer.

In [17]:
algorithms = [label_smoothing, blurpool, prog_resize]

Now let's instantiate our model, optimizer, logger, and trainer again. No need to instantiate our scheduler again because it's stateless!

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from composer.models import ComposerClassifier


class Block(nn.Module):
    """A ResNet block."""

    def __init__(self, f_in: int, f_out: int, downsample: bool = False):
        super(Block, self).__init__()

        stride = 2 if downsample else 1
        self.conv1 = nn.Conv2d(
            f_in, f_out, kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(f_out)
        self.conv2 = nn.Conv2d(
            f_out, f_out, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(f_out)
        self.relu = nn.ReLU(inplace=True)

        # No parameters for shortcut connections.
        if downsample or f_in != f_out:
            self.shortcut = nn.Sequential(
                nn.Conv2d(f_in, f_out, kernel_size=1, stride=2, bias=False),
                nn.BatchNorm2d(f_out),
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x: torch.Tensor):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return self.relu(out)


class ResNetCIFAR(nn.Module):
    """A residual neural network as originally designed for CIFAR-10."""

    def __init__(self, outputs: int = 10):
        super(ResNetCIFAR, self).__init__()

        depth = 56
        width = 16
        num_blocks = (depth - 2) // 6

        plan = [(width, num_blocks), (2 * width, num_blocks), (4 * width, num_blocks)]

        self.num_classes = outputs

        # Initial convolution.
        current_filters = plan[0][0]
        self.conv = nn.Conv2d(
            3, current_filters, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn = nn.BatchNorm2d(current_filters)
        self.relu = nn.ReLU(inplace=True)

        # The subsequent blocks of the ResNet.
        blocks = []
        for segment_index, (filters, num_blocks) in enumerate(plan):
            for block_index in range(num_blocks):
                downsample = segment_index > 0 and block_index == 0
                blocks.append(Block(current_filters, filters, downsample))
                current_filters = filters

        self.blocks = nn.Sequential(*blocks)

        # Final fc layer. Size = number of filters in last segment.
        self.fc = nn.Linear(plan[-1][0], outputs)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x: torch.Tensor):
        out = self.relu(self.bn(self.conv(x)))
        out = self.blocks(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


model = ComposerClassifier(module=ResNetCIFAR(), num_classes=10)

In [21]:
logger_for_algorithm_run = CometMLLogger()

optimizer = composer.optim.DecoupledSGDW(
    model.parameters(), lr=0.05, momentum=0.9, weight_decay=2.0e-3
)

trainer = composer.trainer.Trainer(
    model=model,
    train_dataloader=train_dataloader,
    eval_dataloader=test_dataloader,
    max_duration=train_epochs,
    optimizers=optimizer,
    schedulers=lr_scheduler,
    device=device,
    loggers=logger_for_algorithm_run,
    algorithms=algorithms,  # Adding algorithms this time!
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/lothiraldan/comet-example-mosaicml-getting-started/2da0b5f6b86f450b811be4c672fb9f48

You are using a high value of `weight_decay=0.002` for the `DecoupledSGDW` optimizer. Are you sure you want to do this? Your model's weights will be multiplied by 0.998 on every step!


And let's get training!

In [22]:
# Now we're cooking with algorithms!
start_time = time.perf_counter()
trainer.fit()
end_time = time.perf_counter()
three_epochs_accelerated_time = end_time - start_time
print(f"It took {three_epochs_accelerated_time:0.4f} seconds to train")

******************************
Config:
blurpool/num_blurconv_layers: 4
blurpool/num_blurpool_layers: 0
composer_commit_hash: None
composer_version: 0.23.5
enabled_algorithms/BlurPool: true
enabled_algorithms/LabelSmoothing: true
enabled_algorithms/ProgressiveResizing: true
node_name: unknown because NODENAME environment variable not set
num_cpus_per_node: 1
num_nodes: 1
rank_zero_seed: 4104564262

******************************


train          Epoch   0:    0%|| 0/49 [00:00<?, ?ba/s]                                                       …

/home/lothiraldan/.virtualenvs/tempenv-0634243152b7b/lib/python3.10/site-packages/composer/loss/loss.py:140: UserWarning: Some targets have less than 1 total probability.
  warnings.warn('Some targets have less than 1 total probability.')
/home/lothiraldan/.virtualenvs/tempenv-0634243152b7b/lib/python3.10/site-packages/composer/core/data_spec.py:37: UserWarning: Cannot split tensor of length 848 into batches of size 1024. As it is smaller, no splitting will be done. This may happen on the last batch of a dataset if it is a smaller size than the microbatch size.
  warnings.warn(


eval           Epoch   0:    0%|| 0/10 [00:00<?, ?ba/s]                                                       …

/home/lothiraldan/.virtualenvs/tempenv-0634243152b7b/lib/python3.10/site-packages/composer/core/data_spec.py:37: UserWarning: Cannot split tensor of length 784 into batches of size 1024. As it is smaller, no splitting will be done. This may happen on the last batch of a dataset if it is a smaller size than the microbatch size.
  warnings.warn(


train          Epoch   1:    0%|| 0/49 [00:00<?, ?ba/s]                                                       …

eval           Epoch   1:    0%|| 0/10 [00:00<?, ?ba/s]                                                       …

train          Epoch   2:    0%|| 0/49 [00:00<?, ?ba/s]                                                       …

eval           Epoch   2:    0%|| 0/10 [00:00<?, ?ba/s]                                                       …

It took 883.9192 seconds to train


Again, the runtime will vary based on the instance, but we found that it took about **0.43x-0.75x** as long to train (a **1.3x-2.3x** speedup, which corresponds to 90-400 seconds) relative to the baseline recipe without augmentations. We also found that validation accuracy was similar for the algorithm-enhanced and baseline recipes.

In [23]:
comet_ml.get_running_experiment().end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : 1720104853-colorful-wolf
COMET INFO:     url                   : https://www.comet.com/lothiraldan/comet-example-mosaicml-getting-started/2da0b5f6b86f450b811be4c672fb9f48
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [15]                               : (1.3527661561965942, 2.825247049331665)
COMET INFO:     loss/train/total [147]                  : (1.3337258100509644, 2.885249614715576)
COMET INFO:     metrics/eval/CrossEntropy [3]           : (1.727405309677124, 2.3719818592071533)
COMET INFO:     metrics/eval/MulticlassAccuracy [3]     : (0.21580000221729279, 0.3781000077724457)
COMET INFO:     metrics/train/MulticlassAccuracy [1